In [2]:
import h5py
import numpy as np
import rasterio
import xarray as xr
from pyproj import transform, Proj
import os

bin_dir = os.getcwd()
pkg_dir = os.path.dirname(bin_dir)
data_dir = os.path.join(pkg_dir, 'tests', 'data', 'ri_exclusions')

In [3]:
path = os.path.join(data_dir, 'exclusions_padus.tif')
with rasterio.open(path, 'r') as file:
    print(file.profile)

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 972, 'height': 1434, 'count': 1, 'crs': CRS.from_wkt('PROJCS["unnamed",GEOGCS["GRS 1980(IUGG, 1980)",DATUM["unknown",SPHEROID["GRS80",6378137,298.257222101],TOWGS84[0,0,0,0,0,0,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",41.08333333333334],PARAMETER["central_meridian",-71.5],PARAMETER["scale_factor",0.99999375],PARAMETER["false_easting",100000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]'), 'transform': Affine(90.0, 0.0, 65848.61752782026,
       0.0, -90.0, 103948.14381277534), 'blockxsize': 128, 'blockysize': 128, 'tiled': True, 'compress': 'lzw', 'interleave': 'band'}


In [17]:
path = os.path.join(data_dir, 'exclusions_padus.tif')
with xr.open_rasterio(path, ) as f:
    lon = f.coords['x'].values.astype(np.float32)[...]
    lat = f.coords['y'].values.astype(np.float32)[...]

    lon, lat = np.meshgrid(lon, lat)
    lon = lon.flatten()
    lat = lat.flatten()
    lon_1, lat_1 = transform(Proj(f.attrs['crs']),
                         Proj({"init": "epsg:4326"}),
                         lon, lat)

    %time data_1 = f.values[0, :10, :10]
    slices = (0, slice(None, 10), slice(None, 10))
    %time data_2 = f.values[slices]

np.array_equal(data_1, data_2)

CPU times: user 11.5 ms, sys: 2.32 ms, total: 13.8 ms
Wall time: 14.4 ms
CPU times: user 51 µs, sys: 3 µs, total: 54 µs
Wall time: 55.1 µs


True

In [19]:
data_1.shape

(10, 10)